## Replication for "Asymmetry by Design", October 22, 2020
### By Adam Goldstein and Charlie Eaton

### Table 5: Coefficients for regression of multi-brand strategy on ownership forms

Install STATA packages

In [1]:
*ssc install estout
*ssc install reghdfe

Download datasets

In [2]:
*

In [3]:
quietly {
quietly use data/d_unitidasymmetry, clear
qui keep if year>1991 & year<2016

est clear
    
quietly collapse (rawsum) sftesale sftetotl (mean) all_under system_under grad_rate_150_p_w ///
loan_amount_borrower_c_w tuitionall_c_w selective white_share_w black_share_w hisp_share_w ///
pell_grants_per_fte_c_w ft_faculty_per_100fte_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz online ///
(first) state_n (min) iclevel (max)investor law_enf_frst_this_yr multi_brand, by(systemid year)

xtset systemid year

quietly eststo:reg multi_brand investor system_under online selective ib3.iclevel ///
dpcttype* i.year i.state_n, cluster(systemid) ro

gen multiyear=year if multi_brand==1
bysort systemid: egen minmultiyear=min(multiyear)
replace multi_brand=. if year>minmultiyear

eststo: reghdfe multi_brand investor system_under online selective ib3.iclevel ///
dpcttype*, absorb(systemid year) cluster(systemid)

quietly label var investor "investor ownership"
quietly label var pell_grants_per_fte_c_w "Pell $ / student"
label var multi_brand "multi-brand"
label var system_under "firm enrollment (1,000s)"
label var all_under "campus enrollment (1,000s)"
label var online "online"
label var selective "selective admissions"
    label drop iclevel
label define iclevel 1 "BA offered" 2 "AA offered" 
    label values iclevel iclevel
label var dpcttype_arts "% degr. art"
label var dpcttype_biz "% degr. biz"
label var dpcttype_culinarycosmetic "% degr. personal service"
label var dpcttype_health "% degr. health"
label var dpcttype_law "% degr. law"
label var dpcttype_tech "% degr. tech"
}

In [4]:
%html
esttab, ///
keep (investor system_under online selective 1.iclevel 2.iclevel dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz _cons) ///
mtitle("Pooled OLS" "Fixed effects") eqlabels(none) collabels(none) ///
stats(r2 r2_p N_clust N, fmt(2 2 %9.0fc %9.0fc)  ///
label("r2" "pseudo r2" "firm clusters" N)) b(%9.2fc) se(%9.2fc) /// 
nogaps star(* 0.05 ** 0.01 *** 0.001) label varwidth(10) html

This front-end or document format cannot display HTML

In [5]:
qui {
    est clear
    quietly use data/d_unitidasymmetry, clear
keep if year>1991 & year<2016
    
quietly collapse (rawsum) sftesale sftetotl (mean) all_under system_under grad_rate_150_p_w ///
loan_amount_borrower_c_w tuitionall_c_w selective white_share_w black_share_w hisp_share_w ///
pell_grants_per_fte_c_w ft_faculty_per_100fte_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz online ///
(first) state_n (min) iclevel (max)investor law_enf_frst_this_yr multi_brand, by(systemid year)

xtset  systemid year
    
gen becomemultibrand=1 if d.multi_brand==1
by systemid: drop if l.becomemultibrand ==1 | l2.becomemultibrand ==1 ///
    | l3.becomemultibrand ==1 | l4.becomemultibrand ==1 | ///
    l5.becomemultibrand ==1 | l6.becomemultibrand ==1 | ///
    l7.becomemultibrand ==1 | l8.becomemultibrand ==1 | ///
    l9.becomemultibrand ==1 | l10.becomemultibrand ==1 | ///
    l11.becomemultibrand ==1 | l12.becomemultibrand ==1 | ///
    l13.becomemultibrand ==1 | l14.becomemultibrand ==1 | ///
    l15.becomemultibrand ==1 | l16.becomemultibrand ==1 | ///
    l17.becomemultibrand ==1 | l18.becomemultibrand ==1 | ///
    l19.becomemultibrand ==1 | l20.becomemultibrand ==1 | ///
    l21.becomemultibrand ==1 | l22.becomemultibrand ==1 | l23.becomemultibrand ==1 

replace becomemultibrand=0 if becomemultibrand==. 

**Express time in terms of time at risk (to deal with left censoring)
by systemid: egen minyear=min(year)
gen time=year-minyear

stset time, id(systemid) failure(multi_brand=1)
sts test investor

*Proportional Odds Discrete Time Hazard Model (i.e. logit model)*Very similar to existing model
eststo: logit multi_brand investor system_under online ///
    selective i.iclevel dpct* ibn.time, or cluster(systemid)

quietly label var investor "investor ownership"
quietly label var pell_grants_per_fte_c_w "Pell $ / student"
label var multi_brand "multi-brand"
label var system_under "firm enrollment (1,000s)"
label var all_under "campus enrollment (1,000s)"
label var online "online"
label var selective "selective admissions"
    label drop iclevel
label define iclevel 1 "BA offered" 2 "AA offered" 
    label values iclevel iclevel
label var dpcttype_arts "% degr. art"
label var dpcttype_biz "% degr. biz"
label var dpcttype_culinarycosmetic "% degr. personal service"
label var dpcttype_health "% degr. health"
label var dpcttype_law "% degr. law"
label var dpcttype_tech "% degr. tech"
}

In [6]:
%html
esttab, eform ///
keep (investor system_under online selective 1.iclevel 2.iclevel dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz _cons) ///
mtitle("Proportonal Odds Hazard Model") eqlabels(none) collabels(none) ///
stats(r2 r2_p N_clust N, fmt(2 2 %9.0fc %9.0fc)  ///
label("r2" "pseudo r2" "firm clusters" N)) b(%9.2fc) se(%9.2fc) /// 
nogaps star(* 0.05 ** 0.01 *** 0.001) label varwidth(10) html

This front-end or document format cannot display HTML